<a href="https://colab.research.google.com/github/jhyoo78/jhyoo78/blob/main/log_doc2vec_test_many_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files     # Colab에서 local PC내의 file 읽어 들이는 라이브러리
uploaded = files.upload()          # 파일을 여러 개 선택 가능함

In [ ]:
import numpy as np
from numpy import dot    # cosine similarity 계산 용
from numpy.linalg import norm

import nltk
nltk.download('punkt')   # ‘punkt=문장 tokenizer’를 다운로드
import urllib.request   # 아나콘다에 urllib3 만 있음
#import zipfile           # zipfile36만 있음, 파이선이 3.6 버전이어야 함
from lxml import etree
import re                # 파이썬 정규표현을 위한 표준 라이브러리
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd      # opencv에 conda install로 설치할 것
from gensim.models import Word2Vec     # opencv에  설치할 것
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_text1 = []     # 또는 list()
data_text2 = []

'''------------------------첫번째 train 용 파일 처리---------------------------------'''
f = open('test1.txt', 'r')
lines = f.readlines()
for line in lines:

    sent_text = sent_tokenize(line)   # 각 sentence 를 (예: 'starting rotate log files') 으로 separation 하기
    #print("sent_text=", sent_text)

    # 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환. -----> 전처리 기능 추가할 것
    normalized_text = []

    for string in sent_text:
        tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
        #print("token=", tokens)
        data_text1.append(tokens)
    #print(data_text1)
f.close()
print("data_tex1t =", data_text1)

'''------------------------- test 용 파일 처리 ----------------------------------'''
f = open('test2.txt', 'r')
lines = f.readlines()
for line in lines:

    sent_text = sent_tokenize(line)   # sentence 단위로 (예: 'starting rotate log files') 으로 구분
    #print("sent_text=", sent_text)

    # 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환. -----> 전처리 기능 추가할 것
    normalized_text = []

    for string in sent_text:
        tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
        #print("token=", tokens)
        data_text2.append(tokens)
    #print(data_text1)
f.close()
print("data_text2 = ", data_text2)

'''----------------------------  2개의 taggedDocument 생성-----------------------------'''

# TaggedDocument 생성 --> sentence 별로 index를 추가함.
tagged_data1 = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data_text1)]
tagged_data2 = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data_text2)]

'''----------------------------------- 확인 용 (삭제해도 됨) ----------------------------------------'''
print(tagged_data1)
print(tagged_data2)

print(tagged_data1[0])
#print(tagged_data1[0][1])
#print(tagged_data1[0][1].index('0'))

In [ ]:
# Doc2Vec 모델 생성하기

# Doc2Vec 초기화
model1 = Doc2Vec(vector_size=100, alpha=0.025, min_alpha=0.025, min_count=1, epochs=100, dm =1)   # dm=1이면 PV-DM, 0 이면 PV-DBOW, default epochs=10임
model2 = Doc2Vec(vector_size=100, alpha=0.025, min_alpha=0.025, min_count=1, epochs=100, dm =1)
#  tagged_data의 vocab 구축하기
model1.build_vocab(tagged_data1)    # model의 vocabulary를 tagged_data 라는 이름으로 구축한다.
model2.build_vocab(tagged_data2)
#print(tagged_data1)

# Doc2Vec(tagged_data) 훈련하기
model1.train(tagged_data1, total_examples = model1.corpus_count, epochs = model1.epochs)
model2.train(tagged_data2, total_examples = model2.corpus_count, epochs = model2.epochs)



In [ ]:
# 학습 모델 저장하기
model1.save('log.doc2vec')

In [ ]:
'''------------------------------------ log 분석하기 --------------------------------------------------'''
# Tag(index)를 이용한 유사 sentence 찾기 --- 전처리 과정에서 생략되는 단어/문장도 있으므로 원본 data가 아닌 tagged_data에서 찾아야 함.

'''------------Cosine similarity 계산 함수 ----------------'''
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))
''' -------------------------------------------------------'''

tags = ['146', '209', '910', '1028']   # test1.txt: 146번 sentence => 'init failed run of stage init'   'failed'를 포함한 문장들
topn = 3
faults_sim=[]

'''======================------------------ 고장 메시지와의 유사도 계산 ---------------------=============================='''
for idx, tag1 in enumerate(tags):
    ''' -------------------------- 이하는 확인 용도 (생략 가능) --------------------------------------------------------------------------------'''
    print("target index= ", tag1, " ", "sentence =", tagged_data1[int(tag1)][0])
    similar_sentences_topn = model1.docvecs.most_similar(tag1, topn = topn)  # 훈련된 model1의 tag(index) 로 상위 topn(예: 3개)를 예측한다, 실행할 때마다 결과가 일부 변경됨
    print("similar sentences with index and similarity =", similar_sentences_topn)
    print("")

    for n in range(topn):
        tuple_idx = similar_sentences_topn[n][0]
        tuple_sim = similar_sentences_topn[n][1]
        #print("similar sentence=", "index:", tuple_idx, " ", tuple_sim, " ", tagged_data1[int(tuple_idx)][0])    # 아직 미사용

    print("")

    '''----- 여기에서 for loop로 text2 와 비교하여 유사도를 계산함, test2.txt를 학습하지 않고(model 생성 없이) 비교하면 '없는 문장' 오류 발생 -----'''

    pv1= model1.docvecs[tag1]     # model1의 paragraph vector 추출하기

    rng = len(tagged_data2)  # range
    #print(rng)      # rng= 문장의 수, 그래프에서 x 축으로 표시됨
    flt_sim = []

    for s in range(rng):   #  계산
        pv2 = model2.docvecs[str(s)]  # 시험할 model2의 paragraph vector를 순서대로 추출
        cos_similarity = cos_sim(np.array(pv1), np.array(pv2))  # 두 개 벡터(pv1, pv2)의 Cosine 유사도 계산
        flt_sim.append(cos_similarity)
    faults_sim.append(flt_sim)

print(faults_sim[0])
print(len(faults_sim[1]))


In [ ]:
# 시험 데이터(문장) 각각에 대해 위 4개의 문장 ['146', '209', '910', '1028']에 대한 유사도를 plot함
x_axis = len(tagged_data2)

# 아래 명령의 순서를 바꾸면 빈 그래프도 하나 출력되는 이상한 현상이 나타남.
plt.figure(figsize = (20,10))
plt.axis([ 0, x_axis, 0, 1.2 ])    # x-y 축 지정
plt.plot(np.arange(x_axis), faults_sim[0], 'ro')    # ([x축의 값],[ y 축의 값]) ,  ro => red 문자   ==> for loop로 수정, input window 도 적용, 누적 그래프도....
plt.plot(np.arange(x_axis), faults_sim[1], 'bo')
plt.plot(np.arange(x_axis), faults_sim[2], 'go')
plt.plot(np.arange(x_axis), faults_sim[3], 'yo')
plt.xlabel('sequence number')
plt.ylabel('similarity')
plt.title('Failure Prediction with failure logs')
#plt.savefig('test.png')
plt.show()

# EOF

In [ ]:
''' ---------------------- Cosine 유사도 계산 --------------------------'''
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('doc1과 doc2의 similarity =',cos_sim(doc1, doc2))
print('doc1과 doc3의 similarity =',cos_sim(doc1, doc3))
print('doc2와 doc3의 similarity =',cos_sim(doc2, doc3))   # 벡터의 방향이 같으면 1이 출력됨.
print("")

print('vectors의 유사도 :', cos_sim(np.array(v1), np.array(pv)))

In [ ]:
similar_doc = model1.docvecs.most_similar('146', topn=2)
print(similar_doc)

In [ ]:
pv = model1.docvecs['146']   # tag에 대한 paragraph vector 를 출력, 훈련된 모든 vector의 객체임
print("paragraph vector =", pv)

test_data1 = word_tokenize('init failed run of stage init')    # 데이터에 없는 단어의 embedding, 실행할 때마다 값이 변화함.
v1 = model1.infer_vector(test_data1, alpha=0.025, min_alpha=0.025, epochs=100)
print("Inferred vector=", v1)

test_data2 = word_tokenize('init started on')  # 시험용, tag='146'과 다른 메시지인 경우

#cross_sim = model.docvecs.similarity(tag, str(s))
cross_sim = model1.wv.n_similarity(test_data1, test_data2)   # tokenized words를 입력하여 유사도를 계산함
print(cross_sim)


In [ ]:
''' ================== log file 2개 읽어 처리하기 ==================='''
nums = range(2)
for num in nums:
    path = 'C:/Users/YOO/Desktop/장애예측/Log_exampleafter_pre_processing/'test%d.txt' %num   #=====> Colab에서는 사용 불가
    data = pd.read_csv(path, sep = "\t", , engine='python', encoding = "cp949")
data.head()


아래에서는 PCA를 참고.....

In [ ]:
model = Word2Vec(sentences=token_text, size=10, window=5, min_count=2, workers=4, iter=2, sg=0)   # worker== 사용할 CPU core의 개수
print("model=", model.wv)
print("wor2vector=", model.wv['errorh'])     # 특정 단어를 size 차원의 벡터로 변환하기

model_result = model.wv.most_similar("errorh", topn=10)   # 다른 유사한 단어(상위 10 개)와 유사도 계산
#model_result2 = model.wv.similarity(w1="failed", w2= "out")    # 2개 단어 간의 유사도 계산
print("model_result=", model_result)
#print("model_result2=", model_result2)
print('')


'''------------ 학습 결과 그리기 ==> https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=wideeyed&logNo=221349385092 ---------'''
def plot_2d_graph(vocabs, xs, ys):
    plt.figure(figsize=(10, 8))
    plt.scatter(xs, ys, marker = 'o')
    for i, v in enumerate(vocabs):
        plt.annotate(v, xy=(xs[i], ys[i]))

vocabs = model.wv.vocab.keys()
word_vectors_list=[model.wv[v] for v in vocabs]

'''--- 고차원 벡터를 시각화하기 위해서는 2~3차원 벡터로 축소하는 것이 필요하다. PCA는 대표적인 차원 축소 방식 중 하나임------'''
''' -------------------- PCA는 군집을 표현 못하니 t-SNE로 바꾸어 볼 것 ---------------------'''
from sklearn.decomposition import PCA

pca= PCA(n_components=2)
xys = pca.fit_transform(word_vectors_list)
print("xys=", xys)
print(len(xys))       # xys의 [] 길이는 단어의 개수와 같음, 예 : xys= [[-6.54008631e-02 -1.85528987e-02]....... [-2.234521e-02  3.39717996e-02]]
xs = xys[:, 0]
ys = xys[:, 1]
plot_2d_graph(vocabs, xs, ys)


In [ ]:
print(model.wv.vocab)   # 어휘사전은 단어의 빈도 순으로 출력 ???

In [ ]:
'''---------- 아래 코드는 https://stackoverflow.com/questions/31440803/how-to-fetch-vectors-for-a-word-list-with-word2vec 에서 인용--------- '''
''' ---------- 전체 vectors for word list 를 출력 ---------------'''
my_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    my_dict[key] = model.wv[key]
    # Or my_dict[key] = model.wv.get_vector(key)
    # Or my_dict[key] = model.wv.word_vec(key, use_norm=False)

#%store my_dict > filename.txt   # 파일로 저장하기

In [ ]:
print(my_dict)
# 단어의 수
len(model.wv.vocab)

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

data = ["I love machine learning. Its awesome.",
        "I love coding in python",
        "I love building chatbots",
        "they chat amagingly well"]

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]    # enumerate(x)는 x의 index와 data를 분리한다.

print(tagged_data)

In [ ]:
import numpy as np

x = np.arange(12)  # ==> [0,1,2...11]
x = x.reshape(3,4)

print(x.reshape(-1,1))   # 1차원 배열의 원소를 1개로

print(x.reshape(-1,2))
print(x.reshape(-1,3)) # 1차원 배열의 원소를 3개로

x.reshape(-1)          # 1차원으로 출력